In [117]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import seaborn as sns
import matplotlib.pyplot as plt

In [118]:
df = pd.read_csv("checkins.csv",delimiter=';',low_memory=False)

In [119]:
df = df[df!='                   '].dropna()
df.reset_index(inplace=True,drop=True)
df.head()

F:\Conda\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [120]:
columns = list(df.columns)

In [121]:
columns

['   id    ',
 ' user_id ',
 ' venue_id ',
 '     latitude      ',
 '     longitude     ',
 '     created_at      ']

In [122]:
for i,j in enumerate(columns):
    columns[i] = columns[i].strip()

In [125]:
df.columns = columns

In [126]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [165]:
df_part = df[:100000].iloc[:,[3,4]]

In [166]:
clustering = MeanShift(0.1,n_jobs=-1)
clustering.fit(df_part)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=-1, seeds=None)

In [199]:
clasters = pd.Series(clustering.labels_).value_counts().sort_values(ascending=False)

In [207]:
clasters = clasters[clasters>15]
clasters.index

Int64Index([  0,   1,   2,   4,   3,   5,   6,   7,   8,   9,
            ...
            592, 594, 657, 589, 577, 576, 595, 672, 280, 588],
           dtype='int64', length=592)

In [219]:
# Python 3 program to calculate Distance Between Two Points on Earth 
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lat2, lon1, lon2): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

In [226]:
offices = pd.read_csv("carnival_offices.csv",header=None)

In [228]:
offices

,0,1
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [242]:
distance_to_office = pd.DataFrame(clustering.cluster_centers_[[clasters.index]])

F:\Conda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [257]:
distance_to_office[2] = np.zeros(592)

In [258]:
clustering.cluster_centers_[[clasters.index]]

F:\Conda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [  32.59896002,  -96.89536536],
       [  47.29836829, -122.36173786],
       [  38.04118207, -122.15005326]])

In [259]:
s = 0
for i,j in clustering.cluster_centers_[[clasters.index]]:
    mindist = 100000
    for m in range(offices.shape[0]):    
        if distance(i,offices.iloc[m,0],j,offices.iloc[m,1])<mindist:
            mindist = distance(i,offices.iloc[m,0],j,offices.iloc[m,1])
    distance_to_office.iloc[s,2] = mindist
    s+=1

F:\Conda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [272]:
where_to_place = distance_to_office.sort_values(by=2)[:20].drop(2,axis=1)

In [275]:
for i,j in np.array(where_to_place):
    print(i,j)

52.37296399032261 4.892317222580647
-33.86063042857143 151.20477592857145
25.8456722642857 -80.31889059642857
51.502991260887086 -0.12553728870967767
33.8098779552631 -118.14892380690813
25.7858124199675 -80.2179380368254
25.705349721052592 -80.28342873815798
33.8883253427586 -118.04892817172427
26.010098249285683 -80.19999058571432
33.87298601157018 -118.36209114655645
33.97257482142858 -118.16837066666663
33.81730643390889 -117.891249170958
26.138843786842077 -80.33434683684207
33.98393587403844 -118.00740497307689
33.67430265976576 -117.85878926777275
26.120862658633104 -80.1589066802157
34.06039755458241 -118.24870902659876
26.20058464102565 -80.25071612564099
34.03548695312116 -118.43899771946148
33.65089599288026 -117.75207397281613
